In [1]:
# For data
import networkx as nx
from parse import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display, HTML
from IPython.display import Javascript
from string import Template
import json

In [3]:
import os
import pandas as pd

In [4]:
files = []
f = os.listdir('gml_topo/')
for file in f:
    if '.gml' in file:
        files.append(file)
files

['Getnet.gml',
 'BtAsiaPac.gml',
 'BtNorthAmerica.gml',
 'Eenet.gml',
 'Chinanet.gml',
 'Digex.gml',
 'Istar.gml',
 'Belnet2007.gml',
 'Dataxchange.gml',
 'PionierL1.gml',
 'Garr201104.gml',
 'Cesnet201006.gml',
 'Layer42.gml',
 'Belnet2004.gml',
 'Cesnet1997.gml',
 'Ibm.gml',
 'Belnet2006.gml',
 'Atmnet.gml',
 'AsnetAm.gml',
 'Rhnet.gml',
 'Garr200902.gml',
 'Sago.gml',
 'Arn.gml',
 'Arnes.gml',
 'UniC.gml',
 'Nordu1997.gml',
 'Sunet.gml',
 'Gblnet.gml',
 'Highwinds.gml',
 'Telecomserbia.gml',
 'HiberniaUs.gml',
 'VtlWavenet2011.gml',
 'Belnet2009.gml',
 'Gambia.gml',
 'Biznet.gml',
 'Twaren.gml',
 'VisionNet.gml',
 'Garr201111.gml',
 'Garr201102.gml',
 'DeutscheTelekom.gml',
 'Garr201001.gml',
 'Garr201108.gml',
 'Garr201007.gml',
 'Uunet.gml',
 'KentmanFeb2008.gml',
 'Garr201101.gml',
 'BsonetEurope.gml',
 'Bellcanada.gml',
 'Abilene.gml',
 'Vinaren.gml',
 'Globenet.gml',
 'Bellsouth.gml',
 'Garr201003.gml',
 'Azrena.gml',
 'Ulaknet.gml',
 'Janetlense.gml',
 'Rediris.gml',
 'Garr199

In [5]:
data = []
for file in files:
    print(file)
    H = nx.read_graphml('gml_topo/' + file.replace('.gml', '.graphml'))
    data.append([file, len(H.nodes()), len(H.edges())])
#     print(data)

Getnet.gml
BtAsiaPac.gml
BtNorthAmerica.gml
Eenet.gml
Chinanet.gml
Digex.gml
Istar.gml
Belnet2007.gml
Dataxchange.gml
PionierL1.gml
Garr201104.gml
Cesnet201006.gml
Layer42.gml
Belnet2004.gml
Cesnet1997.gml
Ibm.gml
Belnet2006.gml
Atmnet.gml
AsnetAm.gml
Rhnet.gml
Garr200902.gml
Sago.gml
Arn.gml
Arnes.gml
UniC.gml
Nordu1997.gml
Sunet.gml
Gblnet.gml
Highwinds.gml
Telecomserbia.gml
HiberniaUs.gml
VtlWavenet2011.gml
Belnet2009.gml
Gambia.gml
Biznet.gml
Twaren.gml
VisionNet.gml
Garr201111.gml
Garr201102.gml
DeutscheTelekom.gml
Garr201001.gml
Garr201108.gml
Garr201007.gml
Uunet.gml
KentmanFeb2008.gml
Garr201101.gml
BsonetEurope.gml
Bellcanada.gml
Abilene.gml
Vinaren.gml
Globenet.gml
Bellsouth.gml
Garr201003.gml
Azrena.gml
Ulaknet.gml
Janetlense.gml
Rediris.gml
Garr199901.gml
Oteglobe.gml
Garr201008.gml
Renater2006.gml
Sanet.gml
Internetmci.gml
Garr201010.gml
Switch.gml
Garr201105.gml
Syringa.gml
KentmanApr2007.gml
Esnet.gml
Shentel.gml
Colt.gml
HiberniaNireland.gml
UsSignal.gml
Singaren.gml
Ar

In [6]:
data

[['Getnet.gml', 7, 8],
 ['BtAsiaPac.gml', 20, 31],
 ['BtNorthAmerica.gml', 36, 76],
 ['Eenet.gml', 13, 16],
 ['Chinanet.gml', 42, 66],
 ['Digex.gml', 31, 38],
 ['Istar.gml', 23, 23],
 ['Belnet2007.gml', 21, 31],
 ['Dataxchange.gml', 6, 11],
 ['PionierL1.gml', 36, 41],
 ['Garr201104.gml', 59, 83],
 ['Cesnet201006.gml', 52, 63],
 ['Layer42.gml', 6, 7],
 ['Belnet2004.gml', 23, 43],
 ['Cesnet1997.gml', 13, 12],
 ['Ibm.gml', 18, 24],
 ['Belnet2006.gml', 23, 44],
 ['Atmnet.gml', 21, 22],
 ['AsnetAm.gml', 65, 79],
 ['Rhnet.gml', 16, 18],
 ['Garr200902.gml', 54, 71],
 ['Sago.gml', 18, 17],
 ['Arn.gml', 30, 29],
 ['Arnes.gml', 34, 47],
 ['UniC.gml', 25, 29],
 ['Nordu1997.gml', 14, 13],
 ['Sunet.gml', 26, 49],
 ['Gblnet.gml', 8, 7],
 ['Highwinds.gml', 18, 53],
 ['Telecomserbia.gml', 6, 6],
 ['HiberniaUs.gml', 22, 29],
 ['VtlWavenet2011.gml', 92, 96],
 ['Belnet2009.gml', 21, 31],
 ['Gambia.gml', 28, 28],
 ['Biznet.gml', 29, 33],
 ['Twaren.gml', 20, 20],
 ['VisionNet.gml', 24, 23],
 ['Garr201111.g

In [7]:
df = pd.DataFrame(data, columns = ['Name', 'Nodes number', 'Edges number'])
df.sort_values(by=['Nodes number'])

,Name,Nodes number,Edges number
180,Arpanet196912.gml,4,4
242,Renam.gml,5,4
139,Epoch.gml,6,7
29,Telecomserbia.gml,6,6
8,Dataxchange.gml,6,11
...,...,...,...
70,Colt.gml,153,191
122,UsCarrier.gml,158,189
183,DialtelecomCz.gml,193,151
88,Cogentco.gml,197,245


In [8]:
H1=nx.read_graphml('change_topo/cBren.graphml')
H = nx.Graph(H1)
H

In [9]:
len(H.nodes())

37

In [10]:
len(H.edges())

38

In [11]:
for e in H.edges():
    data = H.get_edge_data(e[1], e[0])
    speeds = ['100Mb/s', '200Mb/s', '300Mb/s']
    print(H[e[1]][e[0]])
    H[e[1]][e[0]]["LinkLabel"] = random.choice(speeds)
#     print(data)

{'LinkType': 'Ethernet', 'LinkLabel': '300Mb/s', 'key': 0, 'BackgroundFlow': 135}
{'LinkType': 'Ethernet', 'LinkLabel': '200Mb/s', 'key': 0, 'BackgroundFlow': 2}
{'LinkSpeed': '1', 'LinkLabel': '300Mb/s', 'LinkSpeedUnits': 'G', 'LinkSpeedRaw': 1000000000.0, 'key': 0, 'BackgroundFlow': 126}
{'LinkSpeed': '100', 'LinkLabel': '300Mb/s', 'LinkSpeedUnits': 'M', 'LinkSpeedRaw': 100000000.0, 'key': 0, 'BackgroundFlow': 96}
{'LinkSpeed': '1', 'LinkLabel': '300Mb/s', 'LinkSpeedUnits': 'G', 'LinkSpeedRaw': 1000000000.0, 'key': 0, 'BackgroundFlow': 264}
{'LinkSpeed': '100', 'LinkLabel': '300Mb/s', 'LinkSpeedUnits': 'M', 'LinkSpeedRaw': 100000000.0, 'key': 0, 'BackgroundFlow': 264}
{'LinkType': 'Ethernet', 'LinkLabel': '100Mb/s', 'key': 0, 'BackgroundFlow': 30}
{'LinkType': 'Ethernet', 'LinkLabel': '300Mb/s', 'key': 0, 'BackgroundFlow': 42}
{'LinkType': 'Ethernet', 'LinkSpeed': '1', 'LinkLabel': '300Mb/s', 'LinkSpeedUnits': 'G', 'LinkSpeedRaw': 1000000000.0, 'key': 0, 'BackgroundFlow': 294}
{'Link

In [12]:
# Base for measure is Mb/s
mmnt = {'b/s':1e-6, 'kb/s': 1e-3, 'Mb/s': 1, 'Gb/s':1e+3, 'Tb/s':1e+6}

In [13]:
def unit_measure(strr):
    strr = strr.replace(" ", "")
    i = strr.rfind('b/s') + 3
    strr = strr[:i]
    for m in mmnt.keys():
        if m in strr:
            pattern = '{:d}' + m
            k = parse(pattern, strr)
            if k != None:
                return k[0]*mmnt[m]
    return None

In [14]:
bflows = {}
for e in H.edges():
    data = H.get_edge_data(e[1], e[0])
    if data['LinkLabel'] != 'Ethernet':
        speed = data['LinkLabel']
    else:
        speed = "100Mb/s"
        data['LinkLabel'] = speed
    bandwidth = unit_measure(speed)
    step = bandwidth / 100.0
    back_flow = random.randrange(0.0, bandwidth, step)
    bflows[e] = {'BackgroundFlow': back_flow}
        
nx.set_edge_attributes(H, bflows)
# print("New data in network")
# print(H.edges(data = True))

In [15]:
bflows

{('0', '3'): {'BackgroundFlow': 120},
 ('0', '28'): {'BackgroundFlow': 267},
 ('1', '3'): {'BackgroundFlow': 84},
 ('1', '4'): {'BackgroundFlow': 124},
 ('1', '6'): {'BackgroundFlow': 150},
 ('1', '7'): {'BackgroundFlow': 81},
 ('2', '34'): {'BackgroundFlow': 47},
 ('2', '3'): {'BackgroundFlow': 96},
 ('3', '8'): {'BackgroundFlow': 237},
 ('3', '14'): {'BackgroundFlow': 4},
 ('3', '21'): {'BackgroundFlow': 141},
 ('3', '22'): {'BackgroundFlow': 52},
 ('3', '29'): {'BackgroundFlow': 43},
 ('3', '30'): {'BackgroundFlow': 142},
 ('5', '14'): {'BackgroundFlow': 20},
 ('9', '15'): {'BackgroundFlow': 213},
 ('10', '16'): {'BackgroundFlow': 102},
 ('10', '17'): {'BackgroundFlow': 166},
 ('10', '11'): {'BackgroundFlow': 31},
 ('11', '12'): {'BackgroundFlow': 207},
 ('11', '13'): {'BackgroundFlow': 0},
 ('11', '21'): {'BackgroundFlow': 85},
 ('15', '22'): {'BackgroundFlow': 204},
 ('17', '25'): {'BackgroundFlow': 116},
 ('18', '23'): {'BackgroundFlow': 92},
 ('19', '21'): {'BackgroundFlow': 192

In [16]:
def print_Graph(H):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[0], edge[1])
        perc = data['BackgroundFlow'] / unit_measure(data['LinkLabel'])
        color = 'GREEN'
        if perc <= 0.5:
            color = 'GREEN'
        elif perc <= 0.8:
            color = 'YELLOW'
        elif perc <= 1:
            color = 'RED'
        edges.append({'from': edge[1], 'to': edge[0], 
                      'dashes':False, 'label': (str)(unit_measure(data['LinkLabel'])) + 'Mb/s', 
                      'font':{'align': 'top'},
                      'color': color,
                      'width': 4})
#         else:
#             edges.append({'from': edge[1], 'to': edge[0],
#                           'dashes':True, 'label': '0Mb/s', 
#                           'font':{'align': 'top'},
#                           'color': 'BLACK'})
    return (nodes, edges)

In [17]:
nodes, edges = print_Graph(H)

In [18]:
# Transform the graph into a JSON graph
data = {"nodes":nodes, "edges":edges}
jsonGraph = json.dumps(data, indent=4)

# Send to Javascript
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [20]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('mynetwork');
    
    var options = {
        width: '900px',
        height: '500px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [16]:
%%html
<div id="mynetwork"></div>

In [21]:
Bandwidth = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))
BackFlows = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))

In [22]:
for edge in H.edges(data = True):
    data_edge = H.get_edge_data(edge[0], edge[1])
    bndw = unit_measure(data_edge['LinkLabel'])
    bflw = data_edge['BackgroundFlow']
#     print(Bandwidth.shape)
#     print(edge)
#     print(Bandwidth[0][237])
    Bandwidth[int(edge[1])][int(edge[0])] = bndw
    Bandwidth[int(edge[0])][int(edge[1])] = bndw
    BackFlows[int(edge[1])][int(edge[0])] = bflw
    BackFlows[int(edge[0])][int(edge[1])] = bflw

In [23]:
Bandwidth.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,0,0,200,0,0,0,0,0,0,...,0,200,0,0,0,0,0,0,0,0
1,0,0,0,300,100,0,300,200,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,100,0,0
3,200,300,100,0,0,0,0,0,200,0,...,0,0,200,200,0,0,0,0,0,0
4,0,100,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
BackFlows.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,0,0,182,0,0,0,0,0,0,...,0,158,0,0,0,0,0,0,0,0
1,0,0,0,117,48,0,207,80,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,36,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12,0,0
3,182,117,36,0,0,0,0,0,98,0,...,0,0,104,96,0,0,0,0,0,0
4,0,48,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
from itertools import product

def print_answer(status, correct, answ, V, K, M):
    print(status)
    if correct == "correct":
        for i, j, k, t in product(V, V, K, M):
            if (answ[i][j][k][t] != None) and (answ[i][j][k][t] > 0.0):
                print("answer[{}][{}][{}][{}] = {}".format(i, j, k, t, answ[i][j][k][t]))
    else:
        print(correct)

In [173]:
Contracts = [{'volume': 900, 't_s':  60, 't':  60, 'A': 3, 'B': 8},
             {'volume': 100, 't_s':  20, 't':  50, 'A': 7, 'B': 21},
             {'volume': 200, 't_s':  10, 't':  50, 'A': 5, 'B': 26},
             {'volume': 300, 't_s':  0, 't':  50, 'A': 4, 'B': 3},
             {'volume': 100, 't_s':  100, 't':  50, 'A': 9, 'B': 17},
             {'volume': 200, 't_s':  0, 't':  20, 'A': 12, 'B': 19},
             {'volume': 100, 't_s':  50, 't':  50, 'A': 8, 'B': 5},
             {'volume': 100, 't_s':  70, 't':  50, 'A': 26, 'B': 8},
             {'volume': 100, 't_s':  40, 't':  50, 'A': 6, 'B': 30},
             {'volume': 100, 't_s':  30, 't':  50, 'A': 33, 'B': 12}]

In [174]:
def set_time(t0, T, Contracts):
    t = {}
    points = set()
    points.add(t0)
    points.add(T)
    for c in Contracts:
        points.add(c['t_s'])
        points.add(c['t_s'] + c['t'])
    points = sorted(list(points))
    for i in range(len(points) - 1):
        e = points[i + 1] - points[i]
        C = set()
        for c in range(len(Contracts)):
            if points[i] >= Contracts[c]['t_s'] and points[i] < Contracts[c]['t_s'] + Contracts[c]['t']:
                C.add(c)
        t[i] = {'em': e, 'Ctm': C}
    return points, t

In [175]:
mmin = 10000000
mmax = 0
for c in Contracts:
    if c['t_s'] < mmin:
        mmin = c['t_s']
    if c['t_s'] + c['t'] > mmax:
        mmax = c['t_s'] + c['t']

In [176]:
V = set(range(Bandwidth.shape[0])) # число вершин
K = set(range(len(Contracts)))     # количество контрактов К
# t0 = 0                             # начало отсчета t0
# T = 1000                           # конец отсчета в сек
t0 = mmin
T = mmax
timeline, Intervals = set_time(t0, T, Contracts)
M = set(range(len(Intervals)))
timeline, Intervals

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150],
 {0: {'em': 10, 'Ctm': {3, 5}},
  1: {'em': 10, 'Ctm': {2, 3, 5}},
  2: {'em': 10, 'Ctm': {1, 2, 3}},
  3: {'em': 10, 'Ctm': {1, 2, 3, 9}},
  4: {'em': 10, 'Ctm': {1, 2, 3, 8, 9}},
  5: {'em': 10, 'Ctm': {1, 2, 6, 8, 9}},
  6: {'em': 10, 'Ctm': {0, 1, 6, 8, 9}},
  7: {'em': 10, 'Ctm': {0, 6, 7, 8, 9}},
  8: {'em': 10, 'Ctm': {0, 6, 7, 8}},
  9: {'em': 10, 'Ctm': {0, 6, 7}},
  10: {'em': 20, 'Ctm': {0, 4, 7}},
  11: {'em': 30, 'Ctm': {4}}})

In [177]:
import ipynb.fs.full.SolvePyomo as spyomo
import ipynb.fs.full.SolveMIP as mip

In [178]:
status, correct_pyomo, answ_pyomo, time_pyomo = spyomo.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_pyomo)
print_answer(status, correct_pyomo, answ_pyomo, V, K, M)

Total time:  27.345536947250366

Problem: 
- Name: unknown
  Lower bound: 493.0
  Upper bound: 493.0
  Number of objectives: 1
  Number of constraints: 632
  Number of variables: 440
  Number of binary variables: 0
  Number of integer variables: 164280
  Number of nonzeros: 440
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 4.25
  Wallclock time: 4.6
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 4.683449029922485
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

answer[1][3][1][2] = 2.0
answer[1][3][1][3] = 2.0
answer[1][3][1][4] = 2.0
answer[1][3][1][5] = 2.0
answer[1][3][1][6] = 2.0
answer[1][3][3][0] = 6.0
answer[1][3][3][1] = 6.0
answer[1][3][3][2] = 6

In [179]:
def send_toJS(t, K, H, answ):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[1], edge[0])
        if data['LinkLabel'] != 'Ethernet':
            k_flow = 0
            label = ''
            for k in K:
                if answ[int(edge[0])][int(edge[1])][k][t] != 0:
                    k_flow += answ[int(edge[0])][int(edge[1])][k][t]
                    label += ', №' + (str)(k)
                if answ[int(edge[1])][int(edge[0])][k][t] != 0:
                    k_flow += answ[int(edge[1])][int(edge[0])][k][t]
                    label += ', №' + (str)(k)
            color = 'BLUE'
            if k_flow != 0:
                perc = (k_flow + BackFlows[int(edge[0])][int(edge[1])]) / Bandwidth[int(edge[0])][int(edge[1])]
                label += "\n" + (str)(k_flow) + "Mb/s"
                if perc <= 0.5:
                    color = 'GREEN'
                elif perc <= 0.8:
                    color = 'YELLOW'
                elif perc <= 1:
                    color = 'RED'        
            edges.append({'from': edge[1], 'to': edge[0], 
                          'dashes':False, 'label': label[2:],
                          'font':{'align': 'top'},
                          'color': color,
                          'width': 4})
        else:
            edges.append({'from': edge[1], 'to': edge[0],
                          'dashes':True, 'label': '0Mb/s', 
                          'font':{'align': 'top'},
                          'color': 'BLACK'})
            
    # Transform the graph into a JSON graph
    data = {"nodes":nodes, "edges":edges}
    jsonGraph = json.dumps(data, indent=4)

    return jsonGraph

In [180]:
t = 0
jsonGraph = send_toJS(t, K, H, answ_pyomo)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [181]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_pyomo');
    
    var options = {
        width: '1000px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [182]:
%%html
<div id="resultnetwork_pyomo"></div>

In [183]:
status, correct_mip, answ_mip, time_mip = mip.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_mip)
print_answer(status, correct_mip, answ_mip, V, K, M)

Total time:  15.272680044174194
475182
answer[1][3][1][2] = 2.0
answer[1][3][1][3] = 2.0
answer[1][3][1][4] = 2.0
answer[1][3][1][5] = 2.0
answer[1][3][1][6] = 2.0
answer[1][3][3][0] = 6.0
answer[1][3][3][1] = 6.0
answer[1][3][3][2] = 6.0
answer[1][3][3][3] = 6.0
answer[1][3][3][4] = 6.0
answer[1][3][8][4] = 2.0
answer[1][3][8][5] = 2.0
answer[1][3][8][6] = 2.0
answer[1][3][8][7] = 2.0
answer[1][3][8][8] = 2.0
answer[2][3][9][3] = 2.0
answer[2][3][9][4] = 2.0
answer[2][3][9][5] = 2.0
answer[2][3][9][6] = 2.0
answer[2][3][9][7] = 2.0
answer[3][8][0][6] = 15.0
answer[3][8][0][7] = 15.0
answer[3][8][0][8] = 15.0
answer[3][8][0][9] = 15.0
answer[3][8][0][10] = 15.0
answer[3][8][7][7] = 2.0
answer[3][8][7][8] = 2.0
answer[3][8][7][9] = 2.0
answer[3][8][7][10] = 2.0
answer[3][14][6][5] = 2.0
answer[3][14][6][6] = 2.0
answer[3][14][6][7] = 2.0
answer[3][14][6][8] = 2.0
answer[3][14][6][9] = 2.0
answer[3][21][1][2] = 2.0
answer[3][21][1][3] = 2.0
answer[3][21][1][4] = 2.0
answer[3][21][1][5] =

In [184]:
(answ_mip == answ_pyomo).all()

True

In [185]:
t = 0
jsonGraph = send_toJS(t, K, H, answ_mip)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [186]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_mip');
    
    var options = {
        width: '800px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [187]:
%%html
<div id="resultnetwork_mip"></div>